### 初始化摄像头

In [1]:
!echo 'jetson' | sudo -S systemctl restart nvargus-daemon && printf '\n'

[sudo] password for jetson: 


In [2]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)
camera.running = True

### 初始化任务

In [3]:
from imagelist import ImageList

TASK = 'city_1018_1'
global imagelist
imagelist = ImageList(TASK+'/record')

### 收集数据

In [4]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import threading
import time


# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image(width=camera.width, height=camera.height)
recor_button = ipywidgets.Button(description='记录')
show_button = ipywidgets.Button(description='显示视频(影响性能)')
global dl
# def showCamera(c):
#     if (show_button.description == '显示视频(影响性能)'):
#         show_button.description = '关闭视频'
#         dl = traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
#     else:
#         show_button.description = '显示视频(影响性能)'
#         dl.unlink()
        
def showCamera(c):        
    traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
show_button.on_click(showCamera)

# create widgets
count_widget = ipywidgets.IntText(description='count')
global state
state = 1
# manually update counts at initialization
count_widget.value = imagelist.get_count()
def record(camera,count_widget):
    
    while True :
         if (state!=1):
            image = camera.value
            imagelist.save_entry(image)
            count_widget.value = imagelist.get_count()
            time.sleep(0.2)


execute_thread = threading.Thread(target=record, args=( camera,count_widget))

def recordclick(c):
    global state
    if recor_button.description == '记录':
        recor_button.description = '停止'
        state = 0
    else:
        recor_button.description = '记录'
        state = 1
recor_button.on_click(recordclick)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]),
    show_button,
    recor_button,
    count_widget
])

display(data_collection_widget)

execute_thread.start()